In [1]:
import seaborn as sns
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def prepData(country):
    df = pd.read_csv(country + '_edits_monthly_total_ce_nce.csv')
    df = df.drop(columns={'Unnamed: 0'})
    df = df.loc[(df['time'] > '2019-05') & (df['time'] < '2023-06')]
    df['name'] = str(country)
    return df

**remove the meta data pep later** - intial 3 countries were calculated differently in the beginning, therefore different calc here

In [3]:
def dataprepMETA(meta_df, edits_df, coutnry_name):
    meta_df = meta_df.drop(columns = {'Unnamed: 0'})
    edits_df = edits_df.drop(columns = {'Unnamed: 0'})
    meta_df = meta_df.fillna(0)
    edits_df = edits_df.fillna(0)
    #edits_df['time']= pd.to_datetime(edits_df['time'], format='%Y-%m').dt.to_period('M')
    #meta_df['time']= pd.to_datetime(meta_df['time'], format='%Y-%m').dt.to_period('M')

    df = pd.merge(edits_df, meta_df, on = 'time', how = 'left')
    df = df.rename(columns={'CE': 'CE_1', 'NCE': 'NCE_1'})
    df = df.loc[(df['time'] > '2019-05') & (df['time'] < '2023-06')]
    
    df['CE'] = df['CE_1'] + df['Meta']
    df['NCE'] = df['NCE_1'] - df['Meta']

    df['percentage'] = (df['CE'] / df['totalEdits']) * 100

    df['name'] = str(coutnry_name)
    
    return df

are_edits = pd.read_csv('are_edits_monthly_total_ce_nce.csv')
are_edits_meta = pd.read_csv('are_edits_monthly_meta.csv')
ARE = dataprepMETA(are_edits_meta, are_edits, 'ARE')

col_edits = pd.read_csv('COL_edits_monthly_total_ce_nce.csv')
col_edits_meta = pd.read_csv('col_edits_monthly_meta.csv')
COL = dataprepMETA(col_edits_meta, col_edits, 'COL')

idn_edits = pd.read_csv('IDN_edits_monthly_total_ce_nce.csv')
idn_edits_meta = pd.read_csv('idn_edits_monthly_meta.csv')
IDN = dataprepMETA(idn_edits_meta, idn_edits, 'IDN')

In [10]:
#data 
VNM = prepData('VNM')
MYS = prepData('MYS')
CHL = prepData('CHL')
QAT = prepData('QAT')
BHR = prepData('BHR')
GRC = prepData('GRC')
URY = prepData('URY')
DOM = prepData('DOM')
OMN = prepData('OMN')
KWT = prepData('KWT')
ECU = prepData('ECU')
SGP = prepData('SGP')
KHM = prepData('KHM')
MUS = prepData('MUS')
CYP = prepData('CYP')
MMR = prepData('MMR')
ZAF = prepData('ZAF')
MLT = prepData('MLT')
VEN = prepData('VEN')
PHL = prepData('PHL')
PAK = prepData('PAK')
TZA = prepData('TZA')
BRA = prepData('BRA')
IND = prepData('IND')
MEX = prepData('MEX')

In [11]:
frames = [ARE, IDN, COL, DOM, VNM, MYS, CHL, QAT, BHR, GRC, URY, OMN, KWT, ECU, SGP, KHM, MUS, CYP, MMR, ZAF, MLT, VEN,
PHL, PAK, TZA, BRA, IND, MEX]

In [12]:
total = pd.concat(frames)

In [13]:
def calc_tables(df):
    t0 = df.loc[(df['time'] >= '2019-05') & (df['time'] <= '2021-06')].reset_index(drop=True)
    t1 = df.loc[(df['time'] >= '2021-07') & (df['time'] <= '2023-07')].reset_index(drop=True)

    corpo_mean_t0 = t0[['CE', 'name']].groupby('name').mean()
    corpo_mean_t0 = corpo_mean_t0.rename(columns={'CE': 't0_corporate'})
    
    corpo_mean_t1 = t1[['CE', 'name']].groupby('name').mean()
    corpo_mean_t1 = corpo_mean_t1.rename(columns={'CE': 't1_corporate'})
    
    non_corpo_mean_t0 = t0[['NCE', 'name']].groupby('name').mean()
    non_corpo_mean_t0 = non_corpo_mean_t0.rename(columns={'NCE': 't0_non_corporate'})
    
    non_corpo_mean_t1 = t1[['NCE', 'name']].groupby('name').mean()
    non_corpo_mean_t1 = non_corpo_mean_t1.rename(columns={'NCE': 't1_non_corporate'})

    combined = pd.merge(corpo_mean_t0, corpo_mean_t1, on = 'name')
    combined = pd.merge(combined, non_corpo_mean_t0, on = 'name')
    combined = pd.merge(combined, non_corpo_mean_t1, on = 'name')

    combined['difference_corporate'] = combined['t1_corporate'] - combined['t0_corporate']  
    combined['difference_non_corporate'] = combined['t1_non_corporate'] - combined['t0_non_corporate']

    return combined

In [14]:
countries_mean = calc_tables(total)
countries_mean = countries_mean.reset_index()

In [15]:
countries_mean

,name,t0_corporate,t1_corporate,t0_non_corporate,t1_non_corporate,difference_corporate,difference_non_corporate
0,ARE,4185.160000,58250.782609,10861.32,58027.434783,54065.622609,47166.114783
1,BHR,1217.666667,4550.217391,568.68,426.391304,3332.550725,-142.288696
2,BRA,310017.960000,100935.956522,494291.68,390124.826087,-209082.003478,-104166.853913
3,CHL,59946.440000,55304.782609,79473.76,32028.260870,-4641.657391,-47445.499130
4,COL,42379.960000,84064.478261,77345.88,37592.956522,41684.518261,-39752.923478
5,CYP,2205.640000,1399.521739,4870.76,13937.260870,-806.118261,9066.500870
6,DOM,3151.304348,2635.391304,3123.76,4191.086957,-515.913043,1067.326957
7,ECU,7808.400000,6117.652174,13562.24,24361.130435,-1690.747826,10798.890435
8,GRC,50651.600000,48310.956522,83769.16,60325.086957,-2340.643478,-23444.073043
9,IDN,200736.000000,382537.043478,714777.76,292798.695652,181801.043478,-421979.064348


In [16]:
countries_mean.to_csv('fig3_top28_ce_nce_mean_difference.csv', header = True)

#### Tables divided by quarter

In [18]:
t = countries_mean.set_index('name').transpose()

In [21]:
t

name,ARE,BHR,BRA,CHL,COL,CYP,DOM,ECU,GRC,IDN,...,OMN,PAK,PHL,QAT,SGP,TZA,URY,VEN,VNM,ZAF
t0_corporate,4185.160000,1217.666667,310017.960000,59946.440000,42379.960000,2205.640000,3151.304348,7808.400000,50651.600000,200736.000000,...,1999.333333,6425.280000,106451.680000,1669.791667,8313.250000,71942.040000,27463.440000,9805.521739,157361.833333,43814.041667
t1_corporate,58250.782609,4550.217391,100935.956522,55304.782609,84064.478261,1399.521739,2635.391304,6117.652174,48310.956522,382537.043478,...,1811.478261,17779.739130,42947.826087,14569.695652,152.954545,131411.086957,2470.434783,2282.304348,12725.565217,13439.173913
t0_non_corporate,10861.320000,568.680000,494291.680000,79473.760000,77345.880000,4870.760000,3123.760000,13562.240000,83769.160000,714777.760000,...,3096.400000,20888.960000,325175.800000,1599.080000,16254.680000,297886.520000,6502.880000,23336.600000,139935.080000,143765.520000
t1_non_corporate,58027.434783,426.391304,390124.826087,32028.260870,37592.956522,13937.260870,4191.086957,24361.130435,60325.086957,292798.695652,...,3815.000000,39923.217391,129953.434783,7783.913043,10027.565217,159497.260870,4319.043478,16754.130435,42644.565217,94524.739130
difference_corporate,54065.622609,3332.550725,-209082.003478,-4641.657391,41684.518261,-806.118261,-515.913043,-1690.747826,-2340.643478,181801.043478,...,-187.855072,11354.459130,-63503.853913,12899.903986,-8160.295455,59469.046957,-24993.005217,-7523.217391,-144636.268116,-30374.867754
difference_non_corporate,47166.114783,-142.288696,-104166.853913,-47445.499130,-39752.923478,9066.500870,1067.326957,10798.890435,-23444.073043,-421979.064348,...,718.600000,19034.257391,-195222.365217,6184.833043,-6227.114783,-138389.259130,-2183.836522,-6582.469565,-97290.514783,-49240.780870


In [19]:
minus_minus = t[['BRA', 'VNM', 'MYS', 'PHL', 'ZAF', 'URY', 'MMR', 'SGP', 'VEN', 'CHL', 'GRC']]
plus_plus = t[['ARE', 'QAT', 'PAK', 'ECU', 'MLT', 'MUS']]
ce_plus_nce_minus = t[['KHM', 'BHR', 'KWT', 'IND', 'COL', 'TZA', 'IDN']]
ce_minus_nce_plus = t[['MEX', 'CYP', 'DOM', 'OMN']]

In [26]:
minus_minus.transpose().to_csv('minus_minus_mean_difference_28_countries.csv', header = True)
plus_plus.transpose().to_csv('plus_plus_mean_difference_28_countries.csv', header = True)
ce_plus_nce_minus.transpose().to_csv('ce_plus_nce_minus_mean_difference_28_countries.csv', header = True)
ce_minus_nce_plus.transpose().to_csv('ce_minus_nce_plus_mean_difference_28_countries.csv', header = True)

In [27]:
minus_minus = pd.read_csv('minus_minus_mean_difference_28_countries.csv')
plus_plus = pd.read_csv('plus_plus_mean_difference_28_countries.csv')
ce_plus_nce_minus = pd.read_csv('ce_plus_nce_minus_mean_difference_28_countries.csv')
ce_minus_nce_plus = pd.read_csv('ce_minus_nce_plus_mean_difference_28_countries.csv')
